<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Notebooks/Llama2_prod_r1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Llama2 Production

SUMA 26/09/2023

#Instructions

Create Folder for the experiment on personal Google Drive
1.   Load model from HuggingFace
2.   Gain access to model: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
3.   
4.   
5.   Create folder 'data' in project folder
6.   Save folder CrisiLexT26 in folder 'data'
7.   Data sourced from: https://crisislex.org/data-collections.html#CrisisLexT26
3.   Mount Google Drive
4.   Update code with Paths
5.   Pip install llama-cpp-python package
version 0.1.4.9
6.   Documentation for llama-ccp: https://llama-cpp-python.readthedocs.io/en/latest/api-reference/
6.   Load packages llama_ccp and json
7.   Run Code

In [4]:
# Install Transformer package
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [5]:
# Import Packages
from langchain import PromptTemplate,  LLMChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import pandas as pd
import numpy as np
import json
import os
import csv
import time
import re
import matplotlib.pyplot as plt

In [6]:
# Mounting GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.


In [7]:
# Logging onto HuggingFace
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00


In [9]:
# Read (timestamps)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder
def process_folder(folder_path):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'period' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_data = process_folder(subfolder_path)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_p = pd.concat(combined_data, ignore_index=True)


# Remove spaces from column names
combined_df_p.columns = combined_df_p.columns.str.replace(' ', '')

# Rename Columns in prep for left join
combined_df_p.rename(columns={'Tweet-ID': 'Tweet ID'}, inplace=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined timestamps read")


Combined timestamps read


In [10]:
# Read and anontate (tweets)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder and add folder name as label
def process_folder(folder_path, label):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'labeled' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                df['Label'] = label
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        label = subfolder
        subfolder_data = process_folder(subfolder_path, label)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_l = pd.concat(combined_data, ignore_index=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined tweets read")
# working_df.head(10)


Combined tweets read


In [11]:
# Check
combined_df_p.columns
combined_df_p.head()


,Timestamp,Tweet ID,Included(Y/N)
0,Thu Jun 20 03:56:19 +0000 2013,347563538547609602,Y
1,Mon Jul 01 16:10:43 +0000 2013,351734622884855808,Y
2,Mon Jul 01 16:19:46 +0000 2013,351736900391931904,Y
3,Mon Jul 01 16:28:15 +0000 2013,351739035271700482,Y
4,Mon Jul 01 16:38:38 +0000 2013,351741648327294977,Y


In [12]:
# Extract earliest timestamp

# Convert it to datetime first
combined_df_p['Timestamp'] = pd.to_datetime(combined_df_p['Timestamp'])

# Group by 'Tweet-ID' and aggregate to find the earliest timestamp
earliest_timestamps = combined_df_p.groupby('Tweet ID')['Timestamp'].agg('min').reset_index()

# Display or use the resulting DataFrame 'earliest_timestamps'
# print(earliest_timestamps)

In [13]:
# Left join

# Perform a left join on 'Tweet ID'
result_df = combined_df_l.merge(earliest_timestamps, on='Tweet ID', how='left')

# Display
# print(result_df)
# result_df.shape
result_df.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,Timestamp
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013-06-20 12:05:25+00:00
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 17:22:10+00:00
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 18:13:07+00:00
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013-06-20 18:29:19+00:00
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 19:09:50+00:00


In [14]:
# Tokenizer
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# Grab 10 random samples
filtered_df = result_df.query("Label == '2013_Colorado_floods'")
testing_df = filtered_df.sample(n=10, random_state=42)
testing_df.columns = testing_df.columns.str.replace(' ', '')
testing_df['TweetText'] = testing_df['TweetText'].astype(str)


testing_df.head(12)


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp
10507,379918454104408064,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013_Colorado_floods,2013-09-17 10:43:12+00:00
10723,380796468728045568,"$CLF Factbox: Floods shut oil, gas operations ...",Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-19 20:52:07+00:00
10726,380808204420014080,Wow- plowing out the mud from the bike path I ...,Eyewitness,Infrastructure and utilities,Related - but not informative,2013_Colorado_floods,2013-09-19 21:38:45+00:00
10646,380450807746797568,Proud to be playing this all star event in #Lo...,Outsiders,Donations and volunteering,Related - but not informative,2013_Colorado_floods,2013-09-18 21:58:35+00:00
10397,379623338672287744,RT @OWHnews: Colorado flooding: Have your frie...,Media,Affected individuals,Related - but not informative,2013_Colorado_floods,2013-09-16 15:10:31+00:00
10664,380490196464041984,#Jamestown resident says things are disappeari...,Outsiders,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-19 00:35:06+00:00
10612,380337729331933184,RT @EnviroAction: Colorado #fracking site feat...,NGOs,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-18 14:29:15+00:00
10499,379836589691506688,Praying for Colorado because its crazy floodin...,Outsiders,Sympathy and support,Related - but not informative,2013_Colorado_floods,2013-09-17 05:17:54+00:00
10845,381803449840435200,RT @denverpost: Colorado's September floods wi...,Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-22 15:33:30+00:00
10122,378191675941588992,"RT @sciam: Two Dead, Hundreds Evacuated, Cars ...",Media,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-12 16:21:36+00:00


In [16]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [66]:
# Create an empty 'answer' column in testing_df
testing_df['answer'] = ''

# Loop over the data
for index, row in testing_df.iterrows():
    text_in = row['TweetText']

    # Create the prompt for the current row
    prompt = f"Please return the type of crisis for the following snippet. Answer with only the response in one word and without any explanation.\n```\n{text_in}\n```"

    # Generate the answer using LLMChain
    answer = llm_chain.run(prompt)

    # Update the 'answer' column in the DataFrame
    testing_df.at[index, 'answer'] = answer


In [67]:
# Print output
testing_df.head(10)


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp,answer
10507,379918454104408064,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013_Colorado_floods,2013-09-17 10:43:12+00:00,[Crisis] \n\nPlease answer with one word only.
10723,380796468728045568,"$CLF Factbox: Floods shut oil, gas operations ...",Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-19 20:52:07+00:00,Please return the type of crisis for the snip...
10726,380808204420014080,Wow- plowing out the mud from the bike path I ...,Eyewitness,Infrastructure and utilities,Related - but not informative,2013_Colorado_floods,2013-09-19 21:38:45+00:00,\n\nYour response: []
10646,380450807746797568,Proud to be playing this all star event in #Lo...,Outsiders,Donations and volunteering,Related - but not informative,2013_Colorado_floods,2013-09-18 21:58:35+00:00,```\n\nYour response: []\n\nPlease let me kno...
10397,379623338672287744,RT @OWHnews: Colorado flooding: Have your frie...,Media,Affected individuals,Related - but not informative,2013_Colorado_floods,2013-09-16 15:10:31+00:00,[crisis]\n\nPlease select one of the followin...
10664,380490196464041984,#Jamestown resident says things are disappeari...,Outsiders,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-19 00:35:06+00:00,[Crisis]\n\nPlease answer with the following ...
10612,380337729331933184,RT @EnviroAction: Colorado #fracking site feat...,NGOs,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-18 14:29:15+00:00,Please return the type of crisis in one word:...
10499,379836589691506688,Praying for Colorado because its crazy floodin...,Outsiders,Sympathy and support,Related - but not informative,2013_Colorado_floods,2013-09-17 05:17:54+00:00,[Crisis]\n\nPlease answer with the correspond...
10845,381803449840435200,RT @denverpost: Colorado's September floods wi...,Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-22 15:33:30+00:00,[ ] \n\nPlease select one of the following op...
10122,378191675941588992,"RT @sciam: Two Dead, Hundreds Evacuated, Cars ...",Media,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-12 16:21:36+00:00,[ ]\n\nPlease respond with one of the followi...


In [65]:
# Test single

text_in = testing_df.loc[10507, 'TweetText']

# Create the prompt for the current row
prompt = f"Please return the type of crisis (one word max) for the following snippet. Answer in the format without any explanation.\n```\n{text_in}\n```"

# Generate the answer using LLMChain
answer = llm_chain.run(prompt)

# print
print(answer)


 Please return the type of crisis (one word max) for the following snippet: [crisis]
Please answer with the following options:
[] - No aspect crisis term exists
"Flood"
"Hurricane"
"Wildfire"
"Earthquake"
"Pandemic"
"Terrorism"
"CyberAttack"
"NaturalDisaster"
"Other"
Please select one option from the above list.


In [61]:
print(text_in)

MT @COEmergency Colorado 2-1-1 Recovery and Relief Guides http://t.co/VyeHrWp5Un … #COflood #hmrd ^CB


In [39]:
print("Number of rows in testing_df:", len(testing_df))
print("Index values in testing_df:", testing_df.index.tolist())
print("First few rows of testing_df:")
print(testing_df.head())

Number of rows in testing_df: 10
Index values in testing_df: [10507, 10723, 10726, 10646, 10397, 10664, 10612, 10499, 10845, 10122]
First few rows of testing_df:
                  TweetID                                          TweetText  \
10507  379918454104408064  MT @COEmergency Colorado 2-1-1 Recovery and Re...   
10723  380796468728045568  $CLF Factbox: Floods shut oil, gas operations ...   
10726  380808204420014080  Wow- plowing out the mud from the bike path I ...   
10646  380450807746797568  Proud to be playing this all star event in #Lo...   
10397  379623338672287744  RT @OWHnews: Colorado flooding: Have your frie...   

      InformationSource               InformationType  \
10507        Government    Donations and volunteering   
10723             Media  Infrastructure and utilities   
10726        Eyewitness  Infrastructure and utilities   
10646         Outsiders    Donations and volunteering   
10397             Media          Affected individuals   

              